<a href="https://colab.research.google.com/github/Alezura01/TattooRecognition/blob/main/ATCNS_TattooMetrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sto importando il file con tutte le risposte del questionario.

In [ ]:
import json
import requests

#Link of the file with all the answers for the gang tattoos.
github_raw_url = "https://raw.githubusercontent.com/Alezura01/TattooRecognition/main/gangTattoo.jsonl"

response = requests.get(github_raw_url)
response_text = response.text
righe = response_text.splitlines()

tabella = []
gruppi={}

#This "For" will read the file line by line, creating an array per line and, when it encounters something different from "N" or "C", in our case is a number, it adds the array
#to a table and creates an element of the dictionary with the name in the line.
# (for example: 1Bloods, it will create as first element of the dictionary "Bloods":[["C","NC",...],[...],...))
for riga in righe:
  if riga[0]=="N" or riga[0]=="C" :
    tabella.append(riga.split())
  else:
    gruppi[riga[1:]]=tabella
    tabella = []

print(gruppi)



{'Bloods': [['NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'C'], ['C', 'NC', 'NC', 'NC', 'C', 'C', 'NC', 'NC', 'C', 'C'], ['NC', 'NC', 'NC', 'NC', 'C', 'NC', 'NC', 'NC', 'NC', 'C'], ['NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'C'], ['NC', 'C', 'NC', 'NC', 'NC', 'NC', 'NC', 'C', 'C', 'C'], ['NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'C', 'C'], ['NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'C', 'C'], ['NC', 'C', 'NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'C', 'C'], ['C', 'C', 'NC', 'NC', 'C', 'NC', 'NC', 'C', 'NC', 'C'], ['C', 'NC', 'C', 'C', 'NC', 'NC', 'NC', 'NC', 'C', 'NC'], ['NC', 'NC', 'NC', 'C', 'NC', 'NC', 'NC', 'C', 'C', 'C'], ['C', 'NC', 'C', 'NC', 'NC', 'NC', 'NC', 'NC', 'C', 'C']], 'Yakuza': [['C', 'NC', 'C', 'NC', 'C', 'NC', 'NC', 'C', 'C', 'C'], ['NC', 'NC', 'NC', 'NC', 'C', 'NC', 'NC', 'C', 'C', 'C'], ['C', 'C', 'C', 'NC', 'C', 'NC', 'C', 'C', 'C', 'C'], ['C', 'C', 'C', 'NC', 'NC', 'C', 'NC', 'C', 'C', 'C'], ['C', 'NC', 'C', 'NC', 'C', 'C', 'NC', 'NC', '

In the performed questionnaire about gang tattoos, we used ten questions:

1. "What specific symbols are present in this tattoo?"
2. "Is this tattoo from mafia or gang?"
3.  "What does this symbol represent in the context of mafia or gang affiliation?"
4. "Is this tattoo from a *group name* member?"
5. "Where on the body is the tattoo located?"
6. "Is there any signifiance to the placement of this tattoo?"
7. "Which is the gang that could use this tattoo?"
8. "Does this photo include a *subject related to the gang which is not present*"
9. "Is this tattoo colored or not?"
10. "What is the color of the tattoo?"

We are going to measure four metrics:
1. TP(True Positive): the answer given by the model is positive, and the answer is positive, so it is signed as correct;
2. TN(True Negative): the answer given by the model is negative, and the answer is negative, so it is signed as correct;
3. FP(False Positive): the answer given by the model is positive, but in reality it is supposed to be negative, so it is signed as incorrect;
4. FN(False Negative): the answer given by the model is negative, but in reality it is supposed to be positive, so it is signed as incorrect;

In [ ]:
TP = 0
TN = 0
FP = 0
FN = 0

TP_tot = []
TN_tot = []
FP_tot = []
FN_tot = []

We compute the four arrays according to the answers.

In [ ]:
for l in range(10):
  for i in gruppi.values():
    # i=0 : [["C","NC","C",...],[...],...]
      for j in range(len(i)):
      #j=0 : ["C","NC","C",...]
        if l == 3:
          if i[j][l] == 'C':
            TP += 1
          else:
            FN += 1
        elif l == 7:
          if i[j][l] == 'C':
            TN += 1
          else:
            FP += 1
        else:
          if i[j][l] == 'C':
            TP += 1
          else:
            FP += 1
  FN_tot.append(FN)
  TP_tot.append(TP)
  FP_tot.append(FP)
  TN_tot.append(TN)
  FP=0
  FN=0
  TN=0
  TP=0

print("FP_TOT is ", FP_tot)
print("TP_tot is" , TP_tot)
print("FN_tot is", FN_tot)
print("TN_tot is", TN_tot)

FP_TOT is  [24, 43, 32, 0, 20, 49, 59, 24, 8, 10]
TP_tot is [53, 34, 45, 16, 57, 28, 18, 0, 69, 67]
FN_tot is [0, 0, 0, 61, 0, 0, 0, 0, 0, 0]
TN_tot is [0, 0, 0, 0, 0, 0, 0, 53, 0, 0]


Now we'll calculate the accuracy, the precision, the recall and the f1 score.

Accuracy -> ratio of TP and TN to the total number of samples

Precision -> ratio of TP to the sum of TP and FP

Recall -> ratio of TP to the sum of TP and FN

F1 Score -> harmonic mean of precision and recall

In [ ]:
#Total number of samples
tot = sum(TP_tot) + sum(TN_tot) +sum(FP_tot) + sum(FN_tot)
#Accuracy
acc = (sum(TP_tot) + sum(TN_tot))/tot

#Precision
prec = sum(TP_tot)/(sum(TP_tot) + sum(FP_tot))

#Recall
rec = sum(TP_tot)/(sum(TP_tot) + sum(FN_tot))

#F1 Score
f1_score = (2 * prec * rec)/(prec + rec)

print("The accuracy is" , acc)
print("The precision is", prec)
print("The recall is", rec)
print("The F1 Score is", f1_score)


The accuracy is 0.5714285714285714
The precision is 0.5899390243902439
The recall is 0.8638392857142857
The F1 Score is 0.7010869565217391
